<a href="https://colab.research.google.com/github/Abdulla-creator-coder/Test-project/blob/main/Traffic_sign_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/AI/PRAICP-1002-TrafSignDetc.zip"
extract_path = "/content/traffic_data"

# Create extraction folder
os.makedirs(extract_path, exist_ok=True)

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Completed!")

Extraction Completed!


In [ ]:
import os

base_path = "/content/traffic_data"

train_path = os.path.join(base_path, "Train")
test_path = os.path.join(base_path, "Test")

train_csv = os.path.join(base_path, "Train.csv")
test_csv = os.path.join(base_path, "Test.csv")

In [ ]:
import pandas as pd

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

print(train_df.head())

   Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0     27      26       5       5      22      20       20   
1     28      27       5       6      23      22       20   
2     29      26       6       5      24      21       20   
3     28      27       5       6      23      22       20   
4     28      26       5       5      23      21       20   

                             Path  
0  Train/20/00020_00000_00000.png  
1  Train/20/00020_00000_00001.png  
2  Train/20/00020_00000_00002.png  
3  Train/20/00020_00000_00003.png  
4  Train/20/00020_00000_00004.png  


In [ ]:
import cv2
import numpy as np

IMG_SIZE = 32

def load_data(df, folder):
    images = []
    labels = []

    for i in range(len(df)):
        img_path = os.path.join(folder, df.loc[i, 'Path'])

        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        images.append(img)
        labels.append(df.loc[i, 'ClassId'])

    return np.array(images), np.array(labels)

X_train, y_train = load_data(train_df, base_path)
X_test, y_test = load_data(test_df, base_path)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y_train))

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 620,523 (2.37 MB)

 Trainable params: 620,523 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=8,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 57s 46ms/step - accuracy: 0.5898 - loss: 1.4073 - val_accuracy: 0.9036 - val_loss: 0.3920
Epoch 2/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.9028 - loss: 0.3231 - val_accuracy: 0.9328 - val_loss: 0.2581
Epoch 3/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 83s 47ms/step - accuracy: 0.9455 - loss: 0.1800 - val_accuracy: 0.9424 - val_loss: 0.2119
Epoch 4/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 56s 46ms/step - accuracy: 0.9642 - loss: 0.1251 - val_accuracy: 0.9461 - val_loss: 0.1959
Epoch 5/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 80s 45ms/step - accuracy: 0.9706 - loss: 0.0967 - val_accuracy: 0.9533 - val_loss: 0.1841
Epoch 6/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 85s 47ms/step - accuracy: 0.9771 - loss: 0.0744 - val_accuracy: 0.9565 - val_loss: 0.1661
Epoch 7/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 56s 46ms/step - accuracy: 0.9804 - loss: 0.0614 - val_accuracy: 0.9561 - val_loss: 0.1769
Epoch 8/8
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.9849 - loss: 0

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

395/395 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9512 - loss: 0.1893
Test Accuracy: 0.9517022967338562
